In [6]:
import sqlite3

def delete_student(index):
    try:
        # Подключаемся к базе данных
        conn = sqlite3.connect('students.db')
        cursor = conn.cursor()
        
        # Проверяем существование таблицы
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='students'")
        if cursor.fetchone() is None:
            print("Таблица students не найдена")
            return
        
        # Выполняем удаление
        cursor.execute("DELETE FROM students WHERE rowid = ?", (index,))
        conn.commit()
        
        # Проверяем, удалилась ли запись
        if cursor.rowcount == 0:
            print(f"Строка с индексом {index} не найдена")
        else:
            print(f"Строка с индексом {index} успешно удалена")
            
    except sqlite3.Error as e:
        print(f"Ошибка при удалении: {e}")
    finally:
        # Закрываем соединение
        conn.close()

# Пример использования
delete_student(1)  # Удаляем строку с индексом 5


Строка с индексом 1 успешно удалена


In [3]:
import logging
import sqlite3
from datetime import datetime, timedelta

# Настройка логирования
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('bot.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

def init_schedule_db():
    """Инициализация базы данных schedule.db с таблицей lectures."""
    try:
        conn = sqlite3.connect('C:/Users/keyne/Documents/PROG_FOLD/schedule.db')
        c = conn.cursor()
        # Очищаем таблицу, если она существует
        c.execute("DROP TABLE IF EXISTS lectures")
        # Создаём таблицу
        c.execute('''CREATE TABLE lectures
                     (id INTEGER PRIMARY KEY AUTOINCREMENT,
                      school TEXT,
                      lecturer TEXT,
                      topic TEXT,
                      description TEXT,
                      link TEXT,
                      date TEXT,
                      direction TEXT)''')
        conn.commit()
        logger.info("Таблица lectures создана в schedule.db")
    except sqlite3.Error as e:
        logger.error(f"Ошибка при создании таблицы lectures: {e}")
    finally:
        conn.close()

def generate_lecture_dates():
    """Генерация дат для лекций: 2 в текущей неделе, 3 вне её."""
    today = datetime(2025, 4, 18)  # Текущая дата
    dates = [
        (today + timedelta(days=1)).strftime('%Y-%m-%d'),  # 19 апреля 2025
        (today + timedelta(days=3)).strftime('%Y-%m-%d'),  # 21 апреля 2025
        (today + timedelta(days=7)).strftime('%Y-%m-%d'),  # 25 апреля 2025
        (today + timedelta(days=13)).strftime('%Y-%m-%d'), # 1 мая 2025
        (today + timedelta(days=22)).strftime('%Y-%m-%d')  # 10 мая 2025
    ]
    return dates

def create_test_lectures():
    """Создание тестовых лекций для каждой школы и направления."""
    schools = ['ШАР', 'ШМР', 'ШБР', 'ШРИ', 'ШОК', 'ШМЯ']
    directions = {
        'ШБР': ['Java', 'C++', 'Python'],
        'ШМР': ['Android', 'iOS', 'Flutter'],
        'ШАР': [None],
        'ШРИ': [None],
        'ШОК': [None],
        'ШМЯ': [None]
    }
    
    try:
        conn = sqlite3.connect('C:/Users/keyne/Documents/PROG_FOLD/schedule.db')
        c = conn.cursor()
        
        total_lectures = 0
        for school in schools:
            for direction in directions[school]:
                dates = generate_lecture_dates()
                for i in range(5):
                    lecture = (
                        school,
                        f"Лектор {i+1}",
                        f"Тема {i+1} ({school}{'' if direction is None else ' ' + direction})",
                        f"Описание лекции {i+1} для {school}{'' if direction is None else ' ' + direction}",
                        f"http://example.com/lecture_{i+1}",
                        dates[i],
                        direction
                    )
                    c.execute('''INSERT INTO lectures (school, lecturer, topic, description, link, date, direction)
                                 VALUES (?, ?, ?, ?, ?, ?, ?)''', lecture)
                    logger.info(f"Добавлена лекция: school={school}, direction={direction}, date={dates[i]}")
                    total_lectures += 1
        
        conn.commit()
        logger.info(f"Создано {total_lectures} тестовых лекций")
        
        # Проверка содержимого базы
        c.execute("SELECT * FROM lectures")
        lectures = c.fetchall()
        logger.debug(f"Содержимое таблицы lectures: {lectures}")
        
    except sqlite3.Error as e:
        logger.error(f"Ошибка при добавлении лекций: {e}")
    finally:
        conn.close()

if __name__ == "__main__":
    init_schedule_db()
    create_test_lectures()

2025-04-18 14:43:03,350 - __main__ - INFO - Таблица lectures создана в schedule.db
2025-04-18 14:43:03,352 - __main__ - INFO - Добавлена лекция: school=ШАР, direction=None, date=2025-04-19
2025-04-18 14:43:03,353 - __main__ - INFO - Добавлена лекция: school=ШАР, direction=None, date=2025-04-21
2025-04-18 14:43:03,354 - __main__ - INFO - Добавлена лекция: school=ШАР, direction=None, date=2025-04-25
2025-04-18 14:43:03,354 - __main__ - INFO - Добавлена лекция: school=ШАР, direction=None, date=2025-05-01
2025-04-18 14:43:03,355 - __main__ - INFO - Добавлена лекция: school=ШАР, direction=None, date=2025-05-10
2025-04-18 14:43:03,356 - __main__ - INFO - Добавлена лекция: school=ШМР, direction=Android, date=2025-04-19
2025-04-18 14:43:03,356 - __main__ - INFO - Добавлена лекция: school=ШМР, direction=Android, date=2025-04-21
2025-04-18 14:43:03,357 - __main__ - INFO - Добавлена лекция: school=ШМР, direction=Android, date=2025-04-25
2025-04-18 14:43:03,358 - __main__ - INFO - Добавлена лекция